Load the important libraries. Note to plot, we need to unload the miniconda module from the MOOSE environment(this is done in bash right now):

In [1]:
import numpy as np
from timeit import default_timer as timer
from matplotlib import pyplot
import math
import numbapro # We use check_cuda() and vectorize
from numbapro import vectorize, autojit, jit, cuda #vectorize for functions and cuda for cuda-u functions
numbapro.check_cuda()

Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: trial mode expires in 30 days
numbapro:1: ImportWarning: The numbapro package is deprecated in favour of the accelerate package. Please update your code to use equivalent functions from accelerate.


------------------------------libraries detection-------------------------------
Finding cublas
	located at /home/john/anaconda2/lib/libcublas.so.7.0.28
	trying to open library...	ok
Finding cusparse
	located at /home/john/anaconda2/lib/libcusparse.so.7.0.28
	trying to open library...	ok
Finding cufft
	located at /home/john/anaconda2/lib/libcufft.so.7.0.35
	trying to open library...	ok
Finding curand
	located at /home/john/anaconda2/lib/libcurand.so.7.0.28
	trying to open library...	ok
Finding nvvm
	located at /home/john/anaconda2/lib/libnvvm.so.3.0.0
	trying to open library...	ok
	finding libdevice for compute_20...	ok
	finding libdevice for compute_30...	ok
	finding libdevice for compute_35...	ok
-------------------------------hardware detection-------------------------------
Found 1 CUDA devices
id 0        GeForce GTX 870M                              [SUPPORTED]
                      compute capability: 3.0
                           pci device id: 0
                              

True

In [2]:
import numba.cuda  
#this line from nbviewer.ipython.org/github/ContinuumIO/numbapro-examples/blob/master/webinars/2014_06_17/intro_to_gpu_python.ipynb
my_gpu = numba.cuda.get_current_device()
print "Running on GPU:", my_gpu.name
cores_per_capability = {
    1: 8,
    2: 32,
    3: 192,
}
cc = my_gpu.compute_capability
print "Compute capability: ", "%d.%d" % cc, "(Numba requires >= 2.0)"
majorcc = cc[0]
print "Number of streaming multiprocessor:", my_gpu.MULTIPROCESSOR_COUNT
cores_per_multiprocessor = cores_per_capability[majorcc]
print "Number of cores per mutliprocessor:", cores_per_multiprocessor
total_cores = cores_per_multiprocessor * my_gpu.MULTIPROCESSOR_COUNT
print "Number of cores on GPU:", total_cores

Running on GPU: GeForce GTX 870M
Compute capability:  3.0 (Numba requires >= 2.0)
Number of streaming multiprocessor: 7
Number of cores per mutliprocessor: 192
Number of cores on GPU: 1344


In [3]:
def iter_loadtxt(filename, delimiter=',', skiprows=1, dtype=float):
    def iter_func():
        with open(filename, 'r') as infile:
            for _ in range(skiprows):
                next(infile)
            for line in infile:
                line = line.rstrip().split(delimiter)
                for item in line:
                    yield dtype(item)
        iter_loadtxt.rowlength = len(line)

    data = np.fromiter(iter_func(), dtype=dtype)
    data = data.reshape((-1, iter_loadtxt.rowlength))
    return data

In [16]:
#This code will import nodal polar vector values using the above iterative loader, define the GPU vectorized functions, 
# and then perform the correlation.

@vectorize(["float64(float64, float64, float64)"], target='gpu') 
def Rfunct(r0, r1, r2):
    return (r0 * r0 +  r1 * r1 + r2 * r2)**(0.5)

@vectorize(["float64(float64, float64, float64, float64, float64, float64)"], target='gpu') 
def Corrfunct(polar0f, polar1f, polar2f, arg0, arg1, arg2):
    return 1/(0.8 * 0.8) * (polar0f * arg0 + polar1f * arg1 + polar2f * arg2);

for t in range(1,1):
    file_load = iter_loadtxt('/media/john/My Passport/Sphere project data/run2--curl/data'+repr(t)+'.csv',delimiter=',',skiprows=1,dtype=float)
    
    polar0 = np.asarray([row[1] for row in file_load])
    polar1 = np.asarray([row[2] for row in file_load])
    polar2 = np.asarray([row[3] for row in file_load])
     
    polar0ave = np.mean(polar0)
    polar1ave = np.mean(polar1)
    polar2ave = np.mean(polar2)
    
    Polar0fluct = np.asarray(polar0 - polar0ave)
    Polar1fluct = np.asarray(polar1 - polar1ave)
    Polar2fluct = np.asarray(polar2 - polar2ave)
    
    r0 = np.asarray([row[4] for row in file_load])
    r1 = np.asarray([row[5] for row in file_load])
    r2 = np.asarray([row[6] for row in file_load])
    
    Cr = np.zeros(len(polar0));
    R = np.zeros(len(polar0));

    start = timer()
    R = Rfunct(r0, r1, r2)
    for k in xrange(len(polar0)-1):
        Cr += Corrfunct(Polar0fluct, Polar1fluct, Polar2fluct, Polar0fluct[k], Polar1fluct[k], Polar2fluct[k])
    vectoradd_time = timer() - start
    
    np.savetxt("/media/john/My Passport/Sphere project data/run2--curl/Cr_"+repr(t)+".csv", Cr, delimiter=",")
    np.savetxt("/media/john/My Passport/Sphere project data/run2--curl/r_"+repr(t)+".csv", R, delimiter=",")
    
    print("Correlation function took %f seconds" % vectoradd_time)

Correlation function took 6.856306 seconds
Correlation function took 8.636047 seconds
Correlation function took 11.475216 seconds
Correlation function took 15.230894 seconds
Correlation function took 18.516755 seconds
Correlation function took 25.754067 seconds
Correlation function took 31.591189 seconds
Correlation function took 32.404009 seconds
Correlation function took 39.486959 seconds
Correlation function took 49.906812 seconds
Correlation function took 56.423196 seconds
Correlation function took 69.420290 seconds
Correlation function took 91.396321 seconds
Correlation function took 104.393568 seconds
Correlation function took 130.417927 seconds
Correlation function took 176.614432 seconds
Correlation function took 207.624661 seconds
Correlation function took 218.468251 seconds
Correlation function took 225.850659 seconds
Correlation function took 242.397990 seconds
Correlation function took 293.339138 seconds
Correlation function took 324.132494 seconds
Correlation function took

In [7]:
polar0

array([-0.20504 , -0.22127 , -0.21911 , ..., -0.055346, -0.096231, -0.11383 ])

In [5]:
file_load = iter_loadtxt('/media/john/My Passport/Sphere project data/run2--curl/data4.csv',delimiter=',',skiprows=1,dtype=float)
    
polar0 = np.asarray([row[1] for row in file_load])
polar1 = np.asarray([row[2] for row in file_load])
polar2 = np.asarray([row[3] for row in file_load])
     
polar0ave = np.mean(polar0)
polar1ave = np.mean(polar1)
polar2ave = np.mean(polar2)
    
Polar0fluct = np.asarray(polar0 - polar0ave)
Polar1fluct = np.asarray(polar1 - polar1ave)
Polar2fluct = np.asarray(polar2 - polar2ave)
    
r0 = np.asarray([row[4] for row in file_load])
r1 = np.asarray([row[5] for row in file_load])
r2 = np.asarray([row[6] for row in file_load])

array([-0.7439, -0.55  , -1.2109, ..., -5.7475,  4.2625, -1.9002])

In [7]:
@vectorize(["float64(float64, float64, float64)"], target='gpu') 
def Rfunct(r0, r1, r2):
    return (r0 * r0 +  r1 * r1 + r2 * r2)**(0.5)

In [6]:
#distance function
dist = np.zeros((len(polar0),len(polar0)));
                
for j in range(0,len(polar0)):
    for k in range(0,len(polar0)):
        dist[j,k] = ((r0[j] - r0[k])**2 + (r1[j] - r1[k])**2 + (r2[j] - r2[k])**2)**0.5;



#probably could "gpuize" the outer or inner loop here

KeyboardInterrupt: 

In [11]:
np.savetxt("/media/john/My Passport/Sphere project data/run2--curl/distfunct.csv", dist, delimiter=",")

In [8]:
dist[0,5]

0.9661524334125543

In [7]:
@vectorize(["float64(float64, float64, float64, float64, float64, float64)"], target='gpu') 
def distfunct(r0, r1, r2, arg0, arg1, arg2):
    return ((r0 - arg0)**2 + (r1 - arg1)**2 + (r2 - arg2)**2)**0.5;

In [8]:
dist[1]

array([ 0.81082662,  0.        ,  0.7399886 , ...,  5.58453024,
        5.6401178 ,  3.88298307])

In [23]:
polar0[1] #sorted(dist[1]) #

-0.22126999999999999

In [ ]:
#first bin up to get radial distribution function, 
#then use the bins to count the number of polar vectors and save their indices
#then dot them, sum them plot vs r

In [7]:
dist[1]

array([ 0.81082662,  0.        ,  0.7399886 , ...,  5.58453024,
        5.6401178 ,  3.88298307])

In [41]:
#[Nbin, rtest1] = np.histogram(dist[1], bins=50, range=None, normed=False, weights=None, density=None)

# data[1,3][0] is the distance between node 1 and node 3, 
# data[1,3][1] is the value of the dot product of the vector at node 1 and node 3

In [10]:
data[0,0][0]

0.0

In [11]:
data[1,0]

array([ 0.81082662,  0.15475446])

Let's start at one point(index i = 1), histogram the distances as we scan along (index j != 1):

In [24]:
[Nbin, rtest1] = np.histogram(dist_vec, bins = 35, range=None, normed=False, weights=None, density=None)

import matplotlib.pyplot as plt

plt.plot(rtest1[0:len(Nbin)],Nbin,'r.');
plt.ylabel('N', fontsize = 27); plt.xlabel('r [nm]', fontsize = 27);
plt.tick_params(axis='both', which='major', labelsize=20)
plt.tick_params(axis='both', which='minor', labelsize=20)
plt.show()

In [27]:
Nbin

array([  1,   0,   8,   8,  21,  21,  41,  45,  71,  68, 102, 109, 135,
       164, 173, 202, 246, 260, 302, 313, 364, 414, 449, 478, 514, 570,
       605, 663, 748, 716, 815, 715, 604, 352, 332])

In [12]:
len(rtest1)

51

In [26]:
rtest1

array([ 0.        ,  0.21638577,  0.43277153,  0.6491573 ,  0.86554306,
        1.08192883,  1.2983146 ,  1.51470036,  1.73108613,  1.94747189,
        2.16385766,  2.38024343,  2.59662919,  2.81301496,  3.02940072,
        3.24578649,  3.46217226,  3.67855802,  3.89494379,  4.11132955,
        4.32771532,  4.54410109,  4.76048685,  4.97687262,  5.19325838,
        5.40964415,  5.62602992,  5.84241568,  6.05880145,  6.27518721,
        6.49157298,  6.70795875,  6.92434451,  7.14073028,  7.35711604,
        7.57350181])

In [118]:
@vectorize(["float64(float64, float64, float64, float64, float64, float64)"], target='gpu') 
def distfunct(r0, r1, r2, arg0, arg1, arg2):
    return ((r0 - arg0)**2 + (r1 - arg1)**2 + (r2 - arg2)**2)**0.5;

dist = np.zeros((len(polar0),len(polar0)))
for k in xrange(len(r0)):
    dist[k] += distfunct(r0, r1, r2, r0[k], r1[k], r2[k])

data = np.zeros((len(polar0),len(polar0),2)) 

#the below loop is expensive...

#store the dot product and the distance between the two nodes we are dotting

for i in xrange(len(polar0)):
    for j in xrange(len(polar0)):
        if i != j:
            data[i,j] = np.array([dist[i][j] , polar0[i] * polar0[j] + polar1[i] * polar1[j] + polar2[i] * polar2[j]])

In [127]:
Cr

array([  0.64176131,   4.24031238,  10.35242499,  17.8552479 ,
        25.79324831,  33.41351423,  40.24171227,  45.90614272,
        50.13263573,  52.81124012,  53.83949052,  53.22074326,
        50.94222159,  47.08687345,  41.82900158,  35.43135742,
        28.24807567,  20.78531421,  13.32380852,   6.2167789 ])

In [33]:
sum_total[0,:]

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [116]:
(39.69659216+47.74092078+29.29923654+55.29443464)/4

43.00779603

In [4]:
#full algorithm:
for t in range(3,4):
    start_load = timer()
    file_load = iter_loadtxt('/media/john/My Passport/Sphere project data/run2--curl/data'+repr(t)+'.csv',delimiter=',',skiprows=1,dtype=float)
    
    polar0 = np.asarray([row[1] for row in file_load])
    polar1 = np.asarray([row[2] for row in file_load])
    polar2 = np.asarray([row[3] for row in file_load])
     
    polar0ave = np.mean(polar0)
    polar1ave = np.mean(polar1)
    polar2ave = np.mean(polar2)
    
    Polar0fluct = np.asarray(polar0 - polar0ave)
    Polar1fluct = np.asarray(polar1 - polar1ave)
    Polar2fluct = np.asarray(polar2 - polar2ave)
    
    r0 = np.asarray([row[4] for row in file_load])
    r1 = np.asarray([row[5] for row in file_load])
    r2 = np.asarray([row[6] for row in file_load])

    @vectorize(["float64(float64, float64, float64, float64, float64, float64)"], target = 'gpu') 
    def distance_funct(r0, r1, r2, arg0, arg1, arg2):
        return ((r0 - arg0)**2 + (r1 - arg1)**2 + (r2 - arg2)**2)**0.5;
    
    @vectorize(["float64(float64, float64, float64, float64, float64, float64, float64, float64, float64)"], target='gpu') 
    def dotproduct_funct(p0, p1, p2, arg0, arg1, arg2, arg0mean, arg1mean, arg2mean):
        return (p0 - arg0mean) * (arg0 - arg0mean) + (p1 - arg1mean) * (arg1 - arg0mean) + (p2 - arg2mean) * (arg2 - arg0mean);

    dist = np.zeros((len(polar0),len(polar0)))
    dots = np.zeros((len(polar0),len(polar0)))
    
    polar0mean = np.mean(polar0)
    polar1mean = np.mean(polar1)
    polar2mean = np.mean(polar2)
    
    for k in xrange(len(polar0)):
        dist[k] = distance_funct(r0, r1, r2, r0[k], r1[k], r2[k])
        dots[k] = dotproduct_funct(polar0, polar1, polar2, polar0[k], polar1[k], polar2[k], polar0mean, polar1mean, polar2mean)

    bin_number = 20
    
    a = np.amax(dist)
    print("Array max located at %f nm" % a)
    
    r_bins =  np.linspace(0,a+1,bin_number)
    load_time = timer() - start_load
    
    #times to load the above data across our differently sized systems(scales with filesize probably): 
    
    #18.17, 23.013, 29.118, 37.66, 46.793950, 63.81686, 87.457398, 87.112473, 107.036612 ,160.460258, 168.049912, 335.679145 seconds

    print("Loading data and computing dot products and distances took %f seconds" % load_time)
    
    #start_hist_calc = timer()

    #cr = np.zeros((len(polar0),bin_number))
    #v
    #-----This loop right here is the most expensive one----#
    #sum_vec = np.zeros((len(polar0),bin_number))
    
    #for i in range(0,len(polar0)):

        #[Nbin, rtest] = np.histogram(dist[i], bins = bin_number, range=None, normed=False, weights=None, density=None)
    
    #    for n in range(0,bin_number):
    #        for j in range(0,len(polar0)):
    #            if r_bins[n] < dist[i][j] and dist[i][j] < r_bins[n+1]:
    #            #print dist_vec[j], dot_vec[j]
    #               sum_vec[i,n] += dots[i][j]

    #    cr[i,:] = sum_vec[i,:]
    #--------------------------------#    
    #hist_calc_time = timer() - start_hist_calc
    #print("Binning the histograms, sorting data, and calculating the correlation function took %f seconds" %  hist_calc_time)
    
    #start_corr_mean = timer()
    #for n in range(0,len(r_bins)):
    #    Cr[n] = np.mean(cr[0:len(polar0),n])
    #time_to_mean = timer() - start_corr_mean 
    #print("Computing the mean took %f seconds" %  time_to_mean)
 

Array max located at 13.003779 nm
Loading data and computing dot products and distances took 29.802030 seconds


In [10]:
Cr[:]

array([ 0.33442645,  1.97814286,  4.41877282,  6.59776073,  7.74253634,
        7.59261856,  6.14178359,  3.72717723,  0.83220008, -2.04925948,
       -4.5395033 , -6.3145933 , -7.15268705, -6.99326662, -5.90265594,
       -4.06412025, -1.99554793, -0.38635252,  0.        ,  0.        ])

In [94]:
np.min(dist)

0.0

In [16]:
len(r_bins)

20

In [26]:
a = np.linspace(1,100,10)

for n in xrange(10):
    q = distance_funct(a,n)

In [27]:
np.linspace(0,10,1)

array([   0.,   21.,   32.,   43.,   54.,   65.,   76.,   87.,   98.,  109.])

In [ ]:
for t in range(22,30):
    print("Performing correlations of the size %f sphere:" % t) 
    
    start_load = timer()
    file_load = iter_loadtxt('/media/john/My Passport/sphere-project-new/csv/data'+repr(t-1)+'.csv',delimiter=',',skiprows=1,dtype=float)
    
    polar0 = np.asarray([row[10] for row in file_load])
    polar1 = np.asarray([row[11] for row in file_load])
    polar2 = np.asarray([row[12] for row in file_load])
     
    polar0ave = np.mean(polar0)
    polar1ave = np.mean(polar1)
    polar2ave = np.mean(polar2)
    
    Polar0fluct = np.asarray(polar0 - polar0ave)
    Polar1fluct = np.asarray(polar1 - polar1ave)
    Polar2fluct = np.asarray(polar2 - polar2ave)
    
    r0 = np.asarray([row[22] for row in file_load])
    r1 = np.asarray([row[23] for row in file_load])
    r2 = np.asarray([row[24] for row in file_load])

    @vectorize(["float64(float64, float64, float64, float64, float64, float64)"], target = 'gpu') 
    def distance_funct(r0, r1, r2, arg0, arg1, arg2):
        return ((r0 - arg0)**2 + (r1 - arg1)**2 + (r2 - arg2)**2)**0.5;
    
    @vectorize(["float64(float64, float64, float64, float64, float64, float64, float64, float64, float64)"], target='gpu') 
    def dotproduct_funct(p0, p1, p2, arg0, arg1, arg2, arg0mean, arg1mean, arg2mean):
        return (p0 - arg0mean) * (arg0 - arg0mean) + (p1 - arg1mean) * (arg1 - arg0mean) + (p2 - arg2mean) * (arg2 - arg0mean);

    dist = np.zeros((len(polar0),len(polar0)))
    dots = np.zeros((len(polar0),len(polar0)))
    
    polar0mean = np.mean(polar0)
    polar1mean = np.mean(polar1)
    polar2mean = np.mean(polar2)
    
    for k in range(0,len(polar0)):
        dist[k] = distance_funct(r0, r1, r2, r0[k], r1[k], r2[k])
        dots[k] = dotproduct_funct(polar0, polar1, polar2, polar0[k], polar1[k], polar2[k], polar0mean, polar1mean, polar2mean)

    load_time = timer() - start_load
    print("Loading data and computing dot products and distances took %f seconds" % load_time) 
    
    bin_number = 25 + t

    a = np.amax(dist)
    
    print("Array max located at %f nm" % a)
    r_bins =  np.linspace(0, a + 3*(a - 0)/bin_number, bin_number)

    @autojit #the only difference between the serial version of the function and the GPUized version is @autojit!
    def sum2d(argdots, argdist, argrbin):

        M, N = argdots.shape
        Cr = np.zeros(bin_number)
        sum_vec = np.zeros((N,len(argrbin)))
        result = np.zeros((M,len(argrbin)))
        for i in xrange(M):
            for n in xrange(len(argrbin)):
                for j in xrange(N):
                    if argrbin[n] <= argdist[i,j] and argdist[i,j] <= argrbin[n+1]:
                        sum_vec[i,n] += argdots[i,j]
            result[i,:] = sum_vec[i,:]
        
        for n in range(0,len(argrbin)):
            Cr[n] = np.mean(result[0:M,n])

        return Cr


    start = timer() #for bin_number = 20
    CrAve = sum2d(dots,dist,r_bins)
    sumtime = timer() - start
    
    np.savetxt("/media/john/My Passport/sphere-project-new/csv/Cr_gpu_"+repr(t - 1)+".csv", CrAve, delimiter=",")
    np.savetxt("/media/john/My Passport/sphere-project-new/csv/rbins_gpu_"+repr(t - 1)+".csv", r_bins, delimiter=",")
    
    print("Compute time: %f" % sumtime)

Performing correlations of the size 14.000000 sphere:
Loading data and computing dot products and distances took 68.306695 seconds
Array max located at 18.993443 nm
Compute time: 24.294551
Performing correlations of the size 15.000000 sphere:
Loading data and computing dot products and distances took 74.295684 seconds
Array max located at 19.995366 nm
Compute time: 32.118267
Performing correlations of the size 16.000000 sphere:
Loading data and computing dot products and distances took 83.437861 seconds
Array max located at 20.999647 nm
Compute time: 41.138889
Performing correlations of the size 17.000000 sphere:
Loading data and computing dot products and distances took 96.524824 seconds
Array max located at 21.992092 nm
Compute time: 57.678381
Performing correlations of the size 18.000000 sphere:
Loading data and computing dot products and distances took 121.305189 seconds
Array max located at 23.001598 nm
Compute time: 78.981254
Performing correlations of the size 19.000000 sphere:


In [5]:
for t in range(15,26):
    print("Performing correlations of the size %f sphere:" % t) 
    
    start_load = timer()
    file_load = iter_loadtxt('/media/john/My Passport/Sphere project data/run2--curl/data'+repr(t)+'.csv',delimiter=',',skiprows=1,dtype=float)
    
    polar0 = np.asarray([row[1] for row in file_load])
    polar1 = np.asarray([row[2] for row in file_load])
    polar2 = np.asarray([row[3] for row in file_load])
     
    polar0ave = np.mean(polar0)
    polar1ave = np.mean(polar1)
    polar2ave = np.mean(polar2)
    
    Polar0fluct = np.asarray(polar0 - polar0ave)
    Polar1fluct = np.asarray(polar1 - polar1ave)
    Polar2fluct = np.asarray(polar2 - polar2ave)
    
    r0 = np.asarray([row[4] for row in file_load])
    r1 = np.asarray([row[5] for row in file_load])
    r2 = np.asarray([row[6] for row in file_load])

    @vectorize(["float64(float64, float64, float64, float64, float64, float64)"], target = 'gpu') 
    def distance_funct(r0, r1, r2, arg0, arg1, arg2):
        return ((r0 - arg0)**2 + (r1 - arg1)**2 + (r2 - arg2)**2)**0.5;
    
    @vectorize(["float64(float64, float64, float64, float64, float64, float64, float64, float64, float64)"], target='gpu') 
    def dotproduct_funct(p0, p1, p2, arg0, arg1, arg2, arg0mean, arg1mean, arg2mean):
        return (p0 - arg0mean) * (arg0 - arg0mean) + (p1 - arg1mean) * (arg1 - arg0mean) + (p2 - arg2mean) * (arg2 - arg0mean);

    dist = np.zeros((len(polar0),len(polar0)))
    dots = np.zeros((len(polar0),len(polar0)))
    
    polar0mean = np.mean(polar0)
    polar1mean = np.mean(polar1)
    polar2mean = np.mean(polar2)
    
    for k in xrange(len(polar0)):
        dist[k] = distance_funct(r0, r1, r2, r0[k], r1[k], r2[k])
        dots[k] = dotproduct_funct(polar0, polar1, polar2, polar0[k], polar1[k], polar2[k], polar0mean, polar1mean, polar2mean)

    load_time = timer() - start_load
    print("Loading data and computing dot products and distances took %f seconds" % load_time) 
    
    bin_number = 25

    a = np.amax(dist)
    
    print("Array max located at %f nm" % a)
    r_bins =  np.linspace(0, a + 3*(a - 0)/bin_number, bin_number)

    @autojit #the only difference between the serial version of the function and the GPUized version is @autojit!
    def sum2d(argdots, argdist, argrbin):

        M, N = argdots.shape
        Cr = np.zeros(bin_number)
        sum_vec = np.zeros((N,len(argrbin)))
        result = np.zeros((M,len(argrbin)))
        for i in xrange(M):
            for n in xrange(len(argrbin)):
                for j in xrange(N):
                    if argrbin[n] <= argdist[i,j] and argdist[i,j] <= argrbin[n+1]:
                        sum_vec[i,n] += argdots[i,j]
            result[i,:] = sum_vec[i,:]
        
        for n in range(0,len(argrbin)):
            Cr[n] = np.mean(result[0:M,n])

        return Cr


    start = timer() #for bin_number = 20
    CrAve = sum2d(dots,dist,r_bins)
    sumtime = timer() - start
    
    np.savetxt("/media/john/My Passport/Sphere project data/run2--curl/Cr_gpu_"+repr(t)+".csv", CrAve, delimiter=",")
    np.savetxt("/media/john/My Passport/Sphere project data/run2--curl/rbins_gpu_"+repr(t)+".csv", r_bins, delimiter=",")
    
    print("Compute time: %f" % sumtime)

Performing correlations of the size 15.000000 sphere:
Loading data and computing dot products and distances took 343.774257 seconds
Array max located at 24.992829 nm
Compute time: 879.089498
Performing correlations of the size 16.000000 sphere:


MemoryError: 

In [7]:
for t in range(19,20):
    print("Performing correlations of the size %f sphere:" % t) 
    
    dint =  14
    
    start_load = timer()
    file_load = iter_loadtxt('/media/john/My Passport/Sphere project data/run2--curl/data'+repr(t)+'.csv',delimiter=',',skiprows=1,dtype=float)
    
    polar0 = np.asarray([row[1] for row in file_load])
    polar1 = np.asarray([row[2] for row in file_load])
    polar2 = np.asarray([row[3] for row in file_load])
    
    r0 = np.asarray([row[4] for row in file_load])
    r1 = np.asarray([row[5] for row in file_load])
    r2 = np.asarray([row[6] for row in file_load])

    @vectorize(["float64(float64, float64, float64, float64, float64, float64)"], target = 'gpu') 
    def distance_funct(r0, r1, r2, arg0, arg1, arg2):
        return ((r0 - arg0)**2 + (r1 - arg1)**2 + (r2 - arg2)**2)**0.5;
    
    @vectorize(["float64(float64, float64, float64, float64, float64, float64, float64, float64, float64)"], target='gpu') 
    def dotproduct_funct(p0, p1, p2, arg0, arg1, arg2, arg0mean, arg1mean, arg2mean):
        return (p0 - arg0mean) * (arg0 - arg0mean) + (p1 - arg1mean) * (arg1 - arg0mean) + (p2 - arg2mean) * (arg2 - arg0mean);

    dist = np.zeros((len(polar0)/14,len(polar0)/14))
    dots = np.zeros((len(polar0)/14,len(polar0)/14))
    
    polar0mean = np.mean(polar0)
    polar1mean = np.mean(polar1)
    polar2mean = np.mean(polar2)
    
    for k in range(0,len(polar0),14):
        dist[k] = distance_funct(r0, r1, r2, r0[k], r1[k], r2[k])
        dots[k] = dotproduct_funct(polar0, polar1, polar2, polar0[k], polar1[k], polar2[k], polar0mean, polar1mean, polar2mean)

    load_time = timer() - start_load
    print("Loading data and computing dot products and distances took %f seconds" % load_time) 
    
    bin_number = 25

    a = np.amax(dist)
    
    print("Array max located at %f nm" % a)
    r_bins =  np.linspace(0, a + 3*(a - 0)/bin_number, bin_number)

    @autojit #the only difference between the serial version of the function and the GPUized version is @autojit!
    def sum2d(argdots, argdist, argrbin):

        M, N = argdots.shape
        Cr = np.zeros(bin_number)
        sum_vec = np.zeros((N,len(argrbin)))
        result = np.zeros((M,len(argrbin)))
        for i in range(0,M,14):
            for n in xrange(len(argrbin)):
                for j in xrange(0,N,14):
                    if argrbin[n] <= argdist[i,j] and argdist[i,j] <= argrbin[n+1]:
                        sum_vec[i,n] += argdots[i,j]
            result[i,:] = sum_vec[i,:]
        
        for n in range(0,len(argrbin)):
            Cr[n] = np.mean(result[0:len(result),n])

        return Cr

    # we make larger sphere correlations coarser (16 needs _int_ == 3, 17 needs _int_ == 6)
    start = timer() #for bin_number = 20
    CrAve = sum2d(dots,dist,r_bins)
    sumtime = timer() - start
    
    np.savetxt("/media/john/My Passport/Sphere project data/run2--curl/Cr_gpu_"+repr(t)+".csv", CrAve, delimiter=",")
    np.savetxt("/media/john/My Passport/Sphere project data/run2--curl/rbins_gpu_"+repr(t)+".csv", r_bins, delimiter=",")
    
    print("Compute time: %f" % sumtime)

Performing correlations of the size 19.000000 sphere:


ValueError: could not broadcast input array from shape (95405) into shape (6814)

In [80]:
start = timer() #for bin_number = 30
n2 = sum2d(dots,dist,r_bins)
sumtime = timer() - start
print("Time: %f" % sumtime)

Time: 4.198216


In [88]:
n2

array([ 0.04021924,  0.53595062,  1.2793013 ,  2.15336015,  3.06819442,
        3.87449114,  4.47031047,  4.78965736,  4.77619817,  4.46308878,
        3.86202413,  3.04801654,  2.056992  ,  0.9762016 , -0.13397134,
       -1.20776503, -2.17883605, -3.02525056, -3.69282413, -4.15640061,
       -4.39101193, -4.3919297 , -4.1756501 , -3.75079536, -3.1320893 ,
       -2.38620248, -1.60660877, -0.88018183, -0.28448872,  0.        ])

In [81]:
start = timer() #for bin_number = 35
n3 = sum2d(dots,dist,r_bins)
sumtime = timer() - start
print("Time: %f" % sumtime)

Time: 4.288260


In [92]:
import matplotlib.pyplot as plt

plt.plot(r_bins, n3);
plt.ylabel('Cr', fontsize = 27); plt.xlabel('r [nm]', fontsize = 27);
plt.tick_params(axis='both', which='major', labelsize=20)
plt.tick_params(axis='both', which='minor', labelsize=20)
plt.show()

In [48]:
sum_vec = np.zeros((100,20))
result = np.zeros((100,20))
for i in xrange(100):
    for n in xrange(20):
        for j in xrange(100):
            if argrbin[n] <= argdist[i,j] and argdist[i,j] <= argrbin[n]:
                sum_vec[i,n] += argdots[i,j]
    result[i,:] = sum_vec[i,:]

NameError: name 'argrbin' is not defined

In [110]:
np.linspace(0, a + 3*(a - 0)/bin_number, bin_number)

array([  0.        ,   0.55965765,   1.1193153 ,   1.67897296,
         2.23863061,   2.79828826,   3.35794591,   3.91760356,
         4.47726122,   5.03691887,   5.59657652,   6.15623417,
         6.71589182,   7.27554948,   7.83520713,   8.39486478,
         8.95452243,   9.51418008,  10.07383774,  10.63349539,
        11.19315304,  11.75281069,  12.31246834,  12.872126  ,  13.43178365])

In [4]:
for k in range(0,10,2):
    print k

0
2
4
6
8


In [5]:
#this is for the slab... seems to work..

for t in range(0,1):
    print("Performing correlations of the size %f sphere:" % t) 

    dint = 3;
    
    start_load = timer()
    file_load = iter_loadtxt('/media/john/My Passport/slabs/serge_tests/data0.csv',delimiter=',',skiprows=1,dtype=float)
    
    polar0 = np.asarray([row[0] for row in file_load[0:len(file_load[:,0]):dint]])
    polar1 = np.asarray([row[1] for row in file_load[0:len(file_load[:,1]):dint]])
    polar2 = np.asarray([row[2] for row in file_load[0:len(file_load[:,2]):dint]])
    
    r0 = np.asarray([row[5] for row in file_load[0:len(file_load[:,5]):dint]])
    r1 = np.asarray([row[6] for row in file_load[0:len(file_load[:,6]):dint]])
    r2 = np.asarray([row[7] for row in file_load[0:len(file_load[:,7]):dint]])

    @vectorize(["float64(float64, float64, float64, float64, float64, float64)"], target = 'gpu') 
    def distance_funct(r0, r1, r2, arg0, arg1, arg2):
        return ((r0 - arg0)**2 + (r1 - arg1)**2 + (r2 - arg2)**2)**0.5;
    
    @vectorize(["float64(float64, float64, float64, float64, float64, float64, float64, float64, float64)"], target='gpu') 
    def dotproduct_funct(p0, p1, p2, arg0, arg1, arg2, arg0mean, arg1mean, arg2mean):
        return (p0 - arg0mean) * (arg0 - arg0mean) + (p1 - arg1mean) * (arg1 - arg0mean) + (p2 - arg2mean) * (arg2 - arg0mean);

    dist = np.zeros((len(polar0),len(polar0)))
    dots = np.zeros((len(polar0),len(polar0)))
    
    polar0mean = np.mean(polar0)
    polar1mean = np.mean(polar1)
    polar2mean = np.mean(polar2)
    
    for k in range(0,len(polar0)):
        dist[k] = distance_funct(r0, r1, r2, r0[k], r1[k], r2[k])
        #should clear r from memory here is it del r0?
        dots[k] = (1/(0.8*0.8))*dotproduct_funct(polar0, polar1, polar2, polar0[k], polar1[k], polar2[k], polar0mean, polar1mean, polar2mean)
        #should clear p from memory here
    load_time = timer() - start_load
    print("Loading data and computing dot products and distances took %f seconds" % load_time) 
    
    bin_number = 25
    
    a = np.amax(dist)
    
    print("Array max located at %f nm" % a)
    r_bins =  np.linspace(0, a + 3*(a - 0)/bin_number, bin_number)

    @autojit #the only difference between the serial version of the function and the GPUized version is @autojit!
    def sum2d(argdots, argdist, argrbin):

        M, N = argdots.shape
        Cr = np.zeros(bin_number)
        sum_vec = np.zeros((N,len(argrbin)))
        result = np.zeros((M,len(argrbin)))
        for i in range(0,M):
            for n in xrange(len(argrbin)):
                for j in xrange(0,N):
                    if argrbin[n] <= argdist[i,j] and argdist[i,j] <= argrbin[n+1]:
                        sum_vec[i,n] += argdots[i,j]
            result[i,:] = sum_vec[i,:]
        
        for n in range(0,len(argrbin)):
            Cr[n] = np.mean(result[0:len(result),n])

        return Cr

    # we make larger sphere correlations coarser (16 needs _int_ == 3, 17 needs _int_ == 6)
    start = timer() #for bin_number = 20
    CrAve = sum2d(dots,dist,r_bins)
    del polar0, polar1, polar2, r0, r1, r2, polar0mean, polar1mean, polar2mean, dots, dist
    sumtime = timer() - start
    
    np.savetxt("/media/john/My Passport/slabs/serge_tests/Cr_gpu_"+repr(dint)+"_"+repr(t)+"_"+repr(bin_number)+".csv", CrAve, delimiter=",")
    np.savetxt("/media/john/My Passport/slabs/serge_tests/rbins_gpu_"+repr(dint)+"_"+repr(t)+"_"+repr(bin_number)+".csv", r_bins, delimiter=",")
    
    del bin_number
    
    print("Compute time: %f" % sumtime)

Performing correlations of the size 0.000000 sphere:


KeyboardInterrupt: 

In [28]:
for t in range(42,43):
    print("Performing correlations of the size %f sphere:" % t) 
    #RAM-wise we can do dint = 1 up till 20, but then dint = 2 up till 28, 3 up till 32, 4  at 33, 5 at 37, 6 at 40, 7 @41
    dint = 7
    #if t <= 14:
    #    dint = 1
    #if t == 15:
    #    dint = 2
    #if t == 16:
    #    dint = 3
    #t == 17:
    #    dint = 4
    
    start_load = timer()
    file_load = iter_loadtxt('/media/john/My Passport/sphere-project-new/csv/data'+repr(t-1)+'.csv',delimiter=',',skiprows=1,dtype=float)
    
    polar0 = np.asarray([row[10] for row in file_load[0:len(file_load[:,10]):dint]])
    polar1 = np.asarray([row[11] for row in file_load[0:len(file_load[:,11]):dint]])
    polar2 = np.asarray([row[12] for row in file_load[0:len(file_load[:,12]):dint]])
    
    polarmag = (polar0*polar0 + polar1*polar1 + polar2*polar2)**(0.5)
    
    polar0d = polar0 / polarmag
    polar1d = polar1 / polarmag
    polar2d = polar2 / polarmag
    
    r0 = np.asarray([row[22] for row in file_load[0:len(file_load[:,22]):dint]])
    r1 = np.asarray([row[23] for row in file_load[0:len(file_load[:,23]):dint]])
    r2 = np.asarray([row[24] for row in file_load[0:len(file_load[:,24]):dint]])

    @vectorize(["float64(float64, float64, float64, float64, float64, float64)"], target = 'gpu') 
    def distance_funct(r0, r1, r2, arg0, arg1, arg2):
        return ((r0 - arg0)**2 + (r1 - arg1)**2 + (r2 - arg2)**2)**0.5;
    
    @vectorize(["float64(float64, float64, float64, float64, float64, float64, float64, float64, float64)"], target='gpu') 
    def dotproduct_funct(p0, p1, p2, arg0, arg1, arg2, arg0mean, arg1mean, arg2mean):
        return (p0 - arg0mean) * (arg0 - arg0mean) + (p1 - arg1mean) * (arg1 - arg0mean) + (p2 - arg2mean) * (arg2 - arg0mean);

    dist = np.zeros((len(polar0),len(polar0)))
    dots = np.zeros((len(polar0),len(polar0)))
    
    #polarmean = np.zeros(25)
    
    #polar0mean = np.mean(polar0)
    #polar1mean = np.mean(polar1)
    #polar2mean = np.mean(polar2)
    
    polar0mean = 0
    polar1mean = 0
    polar2mean = 0
    
    for k in range(0,len(polar0)):
        dist[k] = distance_funct(r0, r1, r2, r0[k], r1[k], r2[k])
        #should clear r from memory here is it del r0?
        dots[k] = dotproduct_funct(polar0d, polar1d, polar2d, polar0d[k], polar1d[k], polar2d[k], polar0mean, polar1mean, polar2mean)
        #should clear p from memory here
    load_time = timer() - start_load
    print("Loading data and computing dot products and distances took %f seconds" % load_time) 
    
    bin_number = 25 + t
    
    a = np.amax(dist)
    
    print("Array max located at %f nm" % a)
    r_bins =  np.linspace(0, a + 3*(a - 0)/bin_number, bin_number)

    @autojit #the only difference between the serial version of the function and the GPUized version is @autojit!
    def sum2d(argdots, argdist, argrbin):

        M, N = argdots.shape
        Cr = np.zeros(bin_number)
        sum_vec = np.zeros((N,len(argrbin)))
        result = np.zeros((M,len(argrbin)))
        for i in range(0,M):
            for n in xrange(len(argrbin)):
                for j in xrange(0,N):
                    if argrbin[n] <= argdist[i,j] and argdist[i,j] <= argrbin[n+1]:
                        sum_vec[i,n] += argdots[i,j]
            result[i,:] = sum_vec[i,:]
        
        for n in range(0,len(argrbin)):
            Cr[n] = np.mean(result[0:len(result),n])

        return Cr
    
    #polarmean[t] = (polar0mean * polar0mean + polar1mean * polar1mean + polar2mean * polar2mean)**(0.5)
    
    # we make larger sphere correlations coarser (16 needs _int_ == 3, 17 needs _int_ == 6)
    start = timer() #for bin_number = 20
    CrAve = sum2d(dots,dist,r_bins)
    #del polar0, polar1, polar2, r0, r1, r2, polar0mean, polar1mean, polar2mean, dots, dist
    sumtime = timer() - start
    
    np.savetxt("/media/john/My Passport/sphere-project-new/csv/director_Cr_gpu_"+repr(t - 1)+".csv", CrAve, delimiter=",")
    np.savetxt("/media/john/My Passport/sphere-project-new/csv/director_rbins_gpu_"+repr(t - 1)+".csv", r_bins, delimiter=",")
    
    del bin_number, dint
    
    print("Compute time: %f" % sumtime)
print("Complete.")

Performing correlations of the size 42.000000 sphere:
Loading data and computing dot products and distances took 182.350452 seconds
Array max located at 46.960030 nm
Compute time: 205.973151
Complete.


In [18]:
np.mean(polarmag**(0.5))

0.3245981557686804

In [9]:
del polar1

NameError: name 'polar1' is not defined

In [ ]:
del polar2

In [10]:
del r0

NameError: name 'r0' is not defined

In [11]:
del r1, r2

NameError: name 'r1' is not defined

In [25]:
file_load = iter_loadtxt('/media/john/My Passport/slabs/serge_tests/data0.csv',delimiter=',',skiprows=1,dtype=float)
    
polar0 = np.asarray([row[0] for row in file_load[0:len(file_load[:,0]):dint]])
polar1 = np.asarray([row[1] for row in file_load[0:len(file_load[:,1]):dint]])
polar2 = np.asarray([row[2] for row in file_load[0:len(file_load[:,2]):dint]])
    
r0 = np.asarray([row[5] for row in file_load[0:len(file_load[:,3]):dint]])
r1 = np.asarray([row[5] for row in file_load[0:len(file_load[:,5]):dint]])
r2 = np.asarray([row[6] for row in file_load[0:len(file_load[:,6]):dint]])

In [26]:

r0

array([-40.   , -40.027, -38.   , ...,  31.   ,  35.   ,  39.   ])

In [24]:
r1

array([-40.   , -40.027, -38.   , ...,  31.   ,  35.   ,  39.   ])

In [35]:
for t in range(19,20):
    print("Performing correlations of the size %f sphere:" % t) 

    dint = 3
    #if t <= 14:
    #    dint = 1
    #if t == 15:
    #    dint = 2
    #if t == 16:
    #    dint = 3
    #t == 17:
    #    dint = 4
    
    start_load = timer()
    file_load = iter_loadtxt('/media/john/My Passport/Sphere project data/run2--curl/data'+repr(t)+'.csv',delimiter=',',skiprows=1,dtype=float)
    
    polar0 = np.asarray([row[0] for row in file_load[0:len(file_load[:,0]):dint]])
    polar1 = np.asarray([row[1] for row in file_load[0:len(file_load[:,1]):dint]])
    polar2 = np.asarray([row[2] for row in file_load[0:len(file_load[:,2]):dint]])
    
    r0 = np.asarray([row[5] for row in file_load[0:len(file_load[:,3]):dint]])
    r1 = np.asarray([row[5] for row in file_load[0:len(file_load[:,5]):dint]])
    r2 = np.asarray([row[6] for row in file_load[0:len(file_load[:,6]):dint]])

    @vectorize(["float64(float64, float64, float64, float64, float64, float64)"], target = 'gpu') 
    def distance_funct(r0, r1, r2, arg0, arg1, arg2):
        return ((r0 - arg0)**2 + (r1 - arg1)**2 + (r2 - arg2)**2)**0.5;
    
    @vectorize(["float64(float64, float64, float64, float64, float64, float64, float64, float64, float64)"], target='gpu') 
    def dotproduct_funct(p0, p1, p2, arg0, arg1, arg2, arg0mean, arg1mean, arg2mean):
        return (p0 - arg0mean) * (arg0 - arg0mean) + (p1 - arg1mean) * (arg1 - arg0mean) + (p2 - arg2mean) * (arg2 - arg0mean);

    dist = np.zeros((len(polar0),len(polar0)))
    dots = np.zeros((len(polar0),len(polar0)))
    
    polar0mean = 0.0
    polar1mean = 0.0
    polar2mean = 0.0

    for k in range(0,len(polar0)):
        dist[k] = distance_funct(r0, r1, r2, r0[k], r1[k], r2[k])
        #should clear r from memory here is it del r0?
        dots[k] = (1/(0.8*0.8))*dotproduct_funct(polar0, polar1, polar2, polar0[k], polar1[k], polar2[k], polar0mean, polar1mean, polar2mean)
        #should clear p from memory here
    load_time = timer() - start_load

    print("Loading data and computing dot products and distances took %f seconds" % load_time) 
    
    bin_number = 25
    
    a = np.amax(dist)
    
    print("Array max located at %f nm" % a)
    r_bins =  np.linspace(0, a + 3*(a - 0)/bin_number, bin_number)

    @autojit #the only difference between the serial version of the function and the GPUized version is @autojit!
    def sum2d(argdots, argdist, argrbin):

        M, N = argdots.shape
        q = len(argrbin)
        Cr = np.zeros(bin_number)
        sum_vec = np.zeros((N,q))
        result = np.zeros((M,q))
        for i in range(0,M):
            for n in xrange(q):
                for j in xrange(0,N):
                    if argrbin[n] <= argdist[i,j] and argdist[i,j] <= argrbin[n+1]:
                        sum_vec[i,n] += argdots[i,j]
            result[i,:] = sum_vec[i,:]
        
        for n in range(0,q):
            Cr[n] = np.mean(result[0:len(result),n])

        return Cr
    print("Correlating %f points:" % len(polar0))
    
    # we make larger sphere correlations coarser (16 needs _int_ == 3, 17 needs _int_ == 6)
    start = timer() #for bin_number = 20
    CrAve = sum2d(dots,dist,r_bins)
    del polar0, polar1, polar2, r0, r1, r2, polar0mean, polar1mean, polar2mean, dots, dist
    sumtime = timer() - start
    
    np.savetxt("/media/john/My Passport/Sphere project data/run2--curl/noAve_Cr_gpu_"+repr(dint)+"_"+repr(t)+"_"+repr(bin_number)+".csv", CrAve, delimiter=",")
    np.savetxt("/media/john/My Passport/Sphere project data/run2--curl/noAve_rbins_gpu_"+repr(dint)+"_"+repr(t)+"_"+repr(bin_number)+".csv", r_bins, delimiter=",")
    
    del bin_number, dint
    
    print("Compute time: %f" % sumtime)
print("Complete.")

Performing correlations of the size 19.000000 sphere:
Loading data and computing dot products and distances took 124.297351 seconds
Array max located at 40.972596 nm
Correlating 31802.000000 points:
Compute time: 50.632598
Complete.


In [6]:
del polar0, polar1, polar2, r0, r1, r2, polar0mean, polar1mean, polar2mean, dots, dist